# Lab 3

Authors: Riley Galante, Jeffrey Taylor, Eric Bernard, Austin Hayden

# Prediction Task and Business Application

... someone else do this

# Data Preparation

In [16]:
import pandas as pd
import numpy as np

df = pd.read_csv('Video_games_esrb_rating.csv')

# perform one-hot encoding of the categorical data we are trying to predict 'esrb_rating'
tmp_df = pd.get_dummies(df.esrb_rating,prefix='rating')
df = pd.concat((df,tmp_df),axis=1) # add back into the dataframe

del df['title'] #get rid of unnecessary data
del df['console']
del df['esrb_rating'] #get rid of original column since its one-hot encoded now

print(df.dtypes)
print("\n{:+^60s}\n".format(""))
print(df.info())

alcohol_reference           int64
animated_blood              int64
blood                       int64
blood_and_gore              int64
cartoon_violence            int64
crude_humor                 int64
drug_reference              int64
fantasy_violence            int64
intense_violence            int64
language                    int64
lyrics                      int64
mature_humor                int64
mild_blood                  int64
mild_cartoon_violence       int64
mild_fantasy_violence       int64
mild_language               int64
mild_lyrics                 int64
mild_suggestive_themes      int64
mild_violence               int64
no_descriptors              int64
nudity                      int64
partial_nudity              int64
sexual_content              int64
sexual_themes               int64
simulated_gambling          int64
strong_janguage             int64
strong_sexual_content       int64
suggestive_themes           int64
use_of_alcohol              int64
use_of_drugs_a

### Final Dataset Description
For this dataset, the first 31 columns are descriptors that go into the esrb rating for a video game. All those colums are binary data, where 1 represents a video game that has that particular attribute and 0 means it does not. Because the esrb rating was in string format, we one-hot encoded it so that the logistic regression would be able to successfully predict this categorical data. This is seen in the rating_E, rating_ET, rating_M, and rating_T columns that were added, representing E for everyone, E 10+, M for Mature, and T for Teen respectively. Additionally, we removed unnecessary information from the dataset that would not help our prediction, like the titles and console of the video games.

### Training and Testing Split

In [30]:
from sklearn.model_selection import ShuffleSplit

E, ET, M, T = df['rating_E'].to_numpy(), df['rating_ET'].to_numpy(), df['rating_M'].to_numpy(), df['rating_T'].to_numpy()

#stack rating encodings into one array for target array
y = np.vstack((E, ET, M, T)).T

#remove target from dataset after its loaded in
del df['rating_E']
del df['rating_ET']
del df['rating_M']
del df['rating_T']

#remaining columns in dataset are used for X
X = df.to_numpy()   

#set up shuffle split using example
num_cv_iterations = 3
num_instances = len(y)
cv_object = ShuffleSplit(n_splits = num_cv_iterations, test_size = 0.2)
                         
print(cv_object)

ShuffleSplit(n_splits=3, random_state=None, test_size=0.2, train_size=None)


The 80/20 train-test split is appropriate for our data set because it is not dependent on ordering or a certain time factor. Regardless of which are chosen for training or testing, all instances in our dataset use just the particular descriptors for that game to determine the rating, nothing more.